# S4 HOW DO NEURONS COMMUNICATE VIA SYNAPSES?
##### Adapted from a similar GENESIS tutorial (Bower and Beeman, 2007) by Charlie Franklin and Henry Chen, converted to notebook by Ziao Chen and Zhenru Chen

### Model Features:
The simulation explores the effects of temporal summation for multiple synaptic inputs. Figure 1 shows a schematic of the biological situation in this tutorial, as well as the model. The model is a three compartment model with two dendrite compartments and a soma (which is really more indicative of an axon or spike initiation zone). The model cell receives synaptic input from the equivalent of two pre-synaptic cells: one excitatory, one inhibitory. Each pre-synaptic cell makes a synaptic contact with both dendrite compartments. The Soma is modeled using a Hodgkin‐Huxley model for sodium and potassium channels.
![image](https://raw.githubusercontent.com/chenziao/Software_Tutorials_Colab/main/S4_Synapses/neuron1.gif)

###Install NEURON
First install NEURON package.

TO RUN CODE CLICK THE PLAY BUTTON ON EVERY CELL OR GO TO THE TOP AND CLICK RUNTIME THEN RUN ALL

In [1]:
!pip install neuron

     |████████████████████████████████| 12.6 MB 223 kB/s 


### Download modfiles from github

In [2]:
import os
from os.path import normpath, sep, join

root = 'Software_Tutorials_Colab'
folder = 'S4_Synapses'
pathlist = normpath(os.getcwd()).split(sep)
if pathlist[-1] != folder:
  rootidx = pathlist.index(root) if root in pathlist else -1
  if rootidx>0:
    os.chdir(join(sep,*pathlist[:rootidx]))
  !git clone https://github.com/chenziao/Software_Tutorials_Colab.git
  os.chdir(join(root,folder))
print(os.getcwd())
%ls

Cloning into 'Software_Tutorials_Colab'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 11), reused 40 (delta 9), pack-reused 0
Unpacking objects: 100% (49/49), done.
/content/Software_Tutorials_Colab/S4_Synapses
alphasynapse1.mod  kalpha.mod  naalpha.mod  S4_Synapses.ipynb
alphasynapse2.mod  leak.mod    neuron1.gif


#### Before running the simulation, you need to compile the mod files only once for the first time.

In [3]:
import os
print(os.system('nrnivmodl')) # compile modfiles. Return 0 for success, 1 for failure.

0


### Run the codes below and answer the 6 questions at the end.

In [4]:
import matplotlib.pyplot as plt
from neuron import h
h.load_file('stdrun.hoc')

h.dt = 0.1 # time step (resolution) of the simulation in ms
h.tstop = 100 # how long to run the simulation in ms
h.v_init= -70 # initial membrane potential in mV

soma = h.Section(name='soma')
soma.diam = 30 # micrometers
soma.L = 30 # micrometers
soma.cm = 1 # membrane capacitance uF/cm2
soma.Ra = 25 # ohm-cm

soma.insert('leak'); soma.insert('naalpha'); soma.insert('kalpha')
soma.el_leak = -58.3; soma.ena = 55; soma.ek = -80 # mV
soma.glbar_leak = 1/5000; soma.gnabar_naalpha = 0.12; soma.gkbar_kalpha = 0.036 # S/cm2

dend = [h.Section(name='dend[%d]'% i) for i in range(2)]

for i in range(2):
    dend[i].L = 100 # micrometers
    dend[i].diam = 2 # micrometers
    dend[i].cm = 1 # membrane capacitance uF/cm2
    dend[i].Ra = 25 # ohm-cm
    
    dend[i].insert('leak')
    dend[i].el_leak = -58.3 # mV
    dend[i].glbar_leak = 1/5000 # S/cm2

dend[0].connect(soma(1),0)
dend[1].connect(dend[0](1),0)
h.topology()


|-|       soma(0-1)
   `|       dend[0](0-1)
     `|       dend[1](0-1)



1.0

In [5]:
# Current clamp
ccl = h.IClamp(soma(0.5))
ccl.delay = 20 #delay in ms
ccl.dur = 40.0 #duration in ms

# Synaptic input event
sourceA = h.NetStim()
sourceB = h.NetStim()
sourceA.noise = 0
sourceB.noise = 0

# Attach synapses
Exca = h.AlphaSynapse1(dend[0](0.5))
Inha = h.AlphaSynapse2(dend[0](0.5))
Excb = h.AlphaSynapse1(dend[1](0.5))
Inhb = h.AlphaSynapse2(dend[1](0.5))

# Source connection to Dendrites
ncl0 = h.NetCon(sourceA, Exca, 1, 0, 1)
ncl1 = h.NetCon(sourceB, Inha, 1, 0, 1)
ncl2 = h.NetCon(sourceA, Excb, 1, 0, 1)
ncl3 = h.NetCon(sourceB, Inhb, 1, 0, 1)

# Artificial for recording input
SourceA = h.IntFire1()
SourceB = h.IntFire1()
SourceA.tau = 1e9
SourceA.refrac = 0.1
SourceA.m = 0
SourceB.tau = 1e9
SourceB.refrac = 0.1
SourceB.m = 0
tempcon1 = h.NetCon(sourceA,SourceA,.5,0,1+(1e-9))
tempcon2 = h.NetCon(sourceB,SourceB,.5,0,1+(1e-9))

# Define vectors for recording variables
t_vec = h.Vector(); Soma_vec = h.Vector()
Dend1_vec = h.Vector(); Dend2_vec = h.Vector()
gna_vec = h.Vector(); gk_vec = h.Vector()
m_vec = h.Vector(); h_vec = h.Vector(); n_vec = h.Vector()
Exca_vec = h.Vector(); Inha_vec = h.Vector()
Excb_vec = h.Vector(); Inhb_vec = h.Vector()
SourceA_vec = h.Vector(); SourceB_vec = h.Vector()

# Record the variables
t_vec.record(h._ref_t); Soma_vec.record(soma(0.5)._ref_v)
Dend1_vec.record(dend[0](0.5)._ref_v); Dend2_vec.record(dend[1](0.5)._ref_v)
gna_vec.record(soma(0.5)._ref_gna_naalpha); gk_vec.record(soma(0.5)._ref_gk_kalpha)
m_vec.record(soma(0.5)._ref_m_naalpha)
h_vec.record(soma(0.5)._ref_h_naalpha)
n_vec.record(soma(0.5)._ref_n_kalpha)
Exca_vec.record(Exca._ref_g); Inha_vec.record(Inha._ref_g)
Excb_vec.record(Excb._ref_g); Inhb_vec.record(Inhb._ref_g)
SourceA_vec.record(SourceA._ref_m); SourceB_vec.record(SourceB._ref_m)

def plot_variables(section):
    fig = plt.figure(figsize=(13,8))
    ax11 = fig.add_subplot(221); ax12 = fig.add_subplot(222)
    ax21 = fig.add_subplot(223); ax22 = fig.add_subplot(224)
    # Membrane potentail & channel/synapse conductance
    if section == 'Dend1':
        ax11.plot(t_vec,Dend1_vec,'k')
        ax11.legend(['Dend1 Vm'],loc=1)
        ax12.plot(t_vec,Exca_vec,'r')
        ax12.plot(t_vec,Inha_vec,'b')
        ax12.legend(['g_Exc','g_Inh'],title='Dend1',loc=1)
        ax12.set_ylim(0,0.01)
        ax12.set_ylabel('S/cm2')
    elif section == 'Dend2':
        ax11.plot(t_vec,Dend2_vec,'k')
        ax11.legend(['Dend2 Vm'],loc=1)
        ax12.plot(t_vec,Excb_vec,'r')
        ax12.plot(t_vec,Inhb_vec,'b')
        ax12.legend(['g_Exc','g_Inh'],title='Dend2',loc=1)
        ax12.set_ylim(0,0.01)
        ax12.set_ylabel('uS/cm2')
    else:
        ax11.plot(t_vec,Soma_vec,'k')
        ax11.legend(['Soma Vm'],loc=1)
        ax12.plot(t_vec,gna_vec,'r')
        ax12.plot(t_vec,gk_vec,'b')
        ax12.legend(['gna','gK'],title='Soma',loc=1)
        ax12.set_ylim(0,0.05)
        ax12.set_ylabel('S/cm2')
    ax11.set_xlim(0,h.tstop);
    ax11.set_ylim(-100,100)
    ax11.set_ylabel('mV');
    ax12.set_xlim(0,h.tstop)
    # Soma activation and inactivation variables
    ax21.plot(t_vec,m_vec,'r')
    ax21.plot(t_vec,h_vec,'orange')
    ax21.plot(t_vec,n_vec,'b')
    ax21.legend(['m','h','n'],title='Soma',loc=1)
    ax21.set_xlim(0,h.tstop);
    ax21.set_ylim(-0.05,1.05)
    ax21.set_xlabel('time (ms)')
    ax21.set_ylabel('probability')
    # Synaptic input
    ax22.plot(t_vec,SourceA_vec.as_numpy()+3,'r')
    ax22.plot(t_vec,SourceB_vec,'b')
    ax22.legend(['Source A','Source B'],title='Synaptic input',loc=1)
    ax22.set_xlim(0,h.tstop);
    ax22.set_ylim(0,8)
    ax22.set_xlabel('time (ms)')
    plt.show()

def activemodel(amp,intvlA,numA,delA,intvlB,numB,delB,WExc1,WInh1,WExc2,WInh2,section):
    ccl.amp = amp
    sourceA.interval = intvlA; sourceA.number = numA; sourceA.start = delA
    sourceB.interval = intvlB; sourceB.number = numB; sourceB.start = delB
    Exca.w = WExc1; Inha.w = WInh1
    Excb.w = WExc2; Inhb.w = WInh2
    
    h.run()
    plt.close('all')
    plot_variables(section)

In [6]:
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display
%matplotlib inline

#default setting
WExc1 = 0; WInh1 = 0
WExc2 = 0; WInh2 = 0
amp = 0.2 # amplitude in nA
num = 5; interval = 10 # ms
delA = 30; delB = 40 #ms
section = 'Soma'

w_reset = widgets.Button(description='Reset',icon='history',button_style='primary')
w_sec = widgets.ToggleButtons(value=section,options=['Soma','Dend1','Dend2'],button_style='info')
w_amp = widgets.FloatText(value=amp,step=0.1)
w_intvlA = widgets.BoundedFloatText(value=interval,min=0,max=h.tstop,step=.1)
w_intvlB = widgets.BoundedFloatText(value=interval,min=0,max=h.tstop,step=.1)
w_numA = widgets.BoundedFloatText(value=num,min=0,max=1e6,step=1)
w_numB = widgets.BoundedFloatText(value=num,min=0,max=1e6,step=1)
w_delA = widgets.FloatText(value=delA,step=1)
w_delB = widgets.FloatText(value=delB,step=1)
w_WExc1 = widgets.FloatText(value=WExc1,step=.1)
w_WInh1 = widgets.FloatText(value=WInh1,step=.1)
w_WExc2 = widgets.FloatText(value=WExc2,step=.1)
w_WInh2 = widgets.FloatText(value=WInh2,step=.1)

def reset_default(*args):
    w_amp.value = amp
    w_intvlA.value = w_intvlB.value = interval
    w_numA.value = w_numB.value = num
    w_delA.value = delA; w_delB.value = delB
    w_WExc1.value = WExc1; w_WInh1.value = WInh1
    w_WExc2.value = WExc2; w_WInh2.value = WInh2
w_reset.on_click(reset_default)

labels = ['Current Clamp Amplitude (nA)','','interval (ms)','number','start (ms)','','Dendrite 1','Dendrite 2',
          'Source A','Excitatory synaptic weight','Source B','Inhibitory synaptic weight','Choose compartment']
Labels = [Label(L) for L in labels]
ui = VBox([ w_reset, HBox([Labels[0],w_amp]), HBox([ VBox(Labels[1:8]),
            VBox([Labels[8],w_intvlA,w_numA,w_delA,Labels[9],w_WExc1,w_WExc2]),
            VBox([Labels[10],w_intvlB,w_numB,w_delB,Labels[11],w_WInh1,w_WInh2]) ],
            layout=Layout(justify_content='space-around')), HBox([Labels[12],w_sec]) ])
out = interactive_output(activemodel,{'amp':w_amp,
                                      'intvlA':w_intvlA,'numA':w_numA,'delA':w_delA,'intvlB':w_intvlB,'numB':w_numB,'delB':w_delB,
                                      'WExc1':w_WExc1,'WInh1':w_WInh1,'WExc2':w_WExc2,'WInh2':w_WInh2,'section':w_sec})

display(ui,out)

Output()

### Questions:
1. In 20 words or less, what is the relationship between the plot of soma Vm and Dendrite 1 (dend1) Vm? (To switch to Dendrite1 Vm plot just click on Dend1 button)  
<br>
2. Set the Current Injection to 0. This can be done by changing the amplitude of Current clamp to 0. Leave soma injection current set to zero for all the remaining questions. You should get a lot of flat lines. Make sure you understand what you've just done.
<br>  
Make sure the stimulus input start time is set to 30 ms or higher (to wait till the variables are all at steady state). Set the synaptic weight for dendrite1 excitatory input to 9. Note that the "Source A interval" is set to 10. Study what you see. Now change the "Source A interval" to 3 to see what happens. Repeat this for "Source A intervals" of 5, 2, 1 and 0.5.
<br>  
In 10 words or less, what principle does the change in Dendrite1 plot indicate? (There is a two-word phrase for this). Also, show a plot for each case.  
<br>  
3. Make a plot of the input-output transfer function. That is, plot output rate vs input spikes rate. For the various cases, make sure that interval*number is a constant at 50 ms for parameters in the Source A.    
<br>  
4. Now set the "Source A interval" to "2", “Source A number” to 25, and change the synaptic weight of Dendrite1 Excitatory input to 9.  Set the synaptic weight for Dendrite1 Inhibitory input to 9. Leave everything else as it was. You should note that the inhibitory input has little effect upon the generation of action potentials.
<br>  
Using the "inhibitory synaptic input for Dendrite1", the three "Source B" (note: this feeds the inhibitory synapses) parameters (delay, number and interval), inhibit (suppress) the middle of the three action potentials produced by "Source A" input. You may not modify any "Source A" parameters, and both the first and last action potentials must remain. Answer the question by stating the parameter values you had to use.  Also, show a plot for every case.  
<br>  
5. Change the weight of inhibitory Input for Dendrite1 to 0 and set inhibitory input for Dendrite2 to 9. For this configuration, is the inhibitory synapse more or less effective at suppressing the middle action potential, compared to the one in Question 4?  Defend your answer with numbers by varying the synaptic weights (go to the first decimal level) used in this configuration and the previous one. Give reasons for the results which you see. Also, show a plot for every case.  
<br>  
6. Reverse the inputs. That is, place the excitatory input on Dendrite2 (leave the weight at 9) and the inhibitory input on Dendrite1. For this configuration, is the inhibitory synapse more or less effective at suppressing the middle action potential?  Defend your answer in the same manner as in QUESTION 5, and explain the differences between this situation and the previous one. Again, show a plot for every case.  